In [289]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor, plot_importance
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
plt.style.use('default')


In [290]:
from sklearn.model_selection import train_test_split

In [291]:
df = pd.read_csv('CarvanaEncoded.csv')

In [292]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [293]:
df.head()

,mileage,Bluetooth,Backup Camera,Infotainment,Screen,Navigation,Hands Free Calling,Heated Seats,Power Seat Controls,Rear Air Vents,...,V6,V8,6-Cyl,city_mpg,highway_mpg,price,year,model_cat,details_cat,make_cat
0,57543,1,1,1,0,0,1,0,1,0,...,0,0,0,26,37,19990,2016,43,74,28
1,51210,0,0,1,0,0,0,0,0,0,...,0,0,0,25,36,16990,2011,92,404,13
2,19216,1,1,1,0,0,0,1,1,0,...,0,0,0,26,38,21990,2016,156,687,10
3,54167,1,0,1,0,0,1,1,1,0,...,0,0,0,24,34,17590,2014,278,16,39
4,11168,1,1,1,0,0,0,0,0,0,...,0,0,0,23,29,23990,2021,124,530,10


In [294]:
test_rows = df[-10:]

In [295]:
df = df[:-10]

In [296]:
X = df.drop(['price'], axis=1)
y = df.price

In [297]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [298]:
xgb = XGBRegressor()

In [299]:
from sklearn.model_selection import RepeatedKFold, cross_val_score

In [300]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(xgb, X_train, y_train, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)

In [301]:
scores = np.absolute(scores)

In [302]:
print('Mean RMSE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

Mean RMSE: 3291.817 (421.907)


In [303]:
xgb.fit(X_train, y_train)

c:\Users\abclay\Desktop\Car-Appraisal-Master\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [304]:
y_preds = xgb.predict(X_test)

In [305]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [306]:
RMSE = np.sqrt(mean_squared_error(y_test, y_preds))
MAE = mean_absolute_error(y_test, y_preds)

In [307]:
RMSE

3356.7293275542515

In [308]:
MAE

1965.3155400310213

In [309]:
final_test_x = test_rows.drop('price', axis=1)

In [310]:
final_test_preds = xgb.predict(final_test_x)

c:\Users\abclay\Desktop\Car-Appraisal-Master\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [311]:
for i in range(len(final_test_x)):
    print(f"For an actual price of {test_rows['price'].iloc[i]} the model predicted {final_test_preds[i]}")
    print("The difference for this prediction is: ", test_rows.price.iloc[i] - final_test_preds[i])
    print("-------")
    print("-------")

For an actual price of 42990 the model predicted 39179.58203125
The difference for this prediction is:  3810.41796875
-------
-------
For an actual price of 17590 the model predicted 18256.169921875
The difference for this prediction is:  -666.169921875
-------
-------
For an actual price of 26990 the model predicted 26094.39453125
The difference for this prediction is:  895.60546875
-------
-------
For an actual price of 43990 the model predicted 46388.94140625
The difference for this prediction is:  -2398.94140625
-------
-------
For an actual price of 29590 the model predicted 30681.416015625
The difference for this prediction is:  -1091.416015625
-------
-------
For an actual price of 14990 the model predicted 16698.421875
The difference for this prediction is:  -1708.421875
-------
-------
For an actual price of 52990 the model predicted 54133.125
The difference for this prediction is:  -1143.125
-------
-------
For an actual price of 20590 the model predicted 18202.6171875
The di

In [312]:
errors =[]
for i in range(len(final_test_preds)):
    difference = test_rows.price.iloc[i] - final_test_preds[i]
    errors.append(difference)

errors = np.absolute(errors)

In [313]:
np.mean(errors)

1813.559375